In [ ]:
import os
data = "./Modular_DeepLearning/food101_3_class/data"
train_path = os.path.join(data,"train")
test_path = os.path.join(data,"test")

In [ ]:
from Modular_DeepLearning.food101_3_class.data_setup import createDataloader
import torchvision, torch

# model_weights = torchvision.models.ResNet101_Weights.DEFAULT
# _model = torchvision.models.resnet101(pretrained=model_weights)

model_weights = torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
_model = torchvision.models.efficientnet_b0(weights=model_weights)

auto_transform = model_weights.transforms()

auto_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [ ]:
train_load, test_load, class_names = createDataloader(train_path, test_path, BTACH_SIZE=32, train_transform=auto_transform, test_transform=auto_transform)

In [ ]:
train_load, test_load, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f9fe5cea8c0>,
 ['pizza', 'steak', 'sushi'])

In [ ]:
# pre tranin model location
# pytoch domain lib
# timm or torch image models
# hugging face
# papers with code

In [ ]:
_model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
from torchinfo import summary
summary(_model, input_size=(32,3,224,224))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [32, 1000]                --
├─Sequential: 1-1                                       [32, 1280, 7, 7]          --
│    └─Conv2dNormActivation: 2-1                        [32, 32, 112, 112]        --
│    │    └─Conv2d: 3-1                                 [32, 32, 112, 112]        864
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        64
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        --
│    └─Sequential: 2-2                                  [32, 16, 112, 112]        --
│    │    └─MBConv: 3-4                                 [32, 16, 112, 112]        1,448
│    └─Sequential: 2-3                                  [32, 24, 56, 56]          --
│    │    └─MBConv: 3-5                                 [32, 24, 56, 56]          6,004
│    │    └─MBConv: 3-6                              

In [ ]:
_model.avgpool

AdaptiveAvgPool2d(output_size=1)

In [ ]:
_model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [ ]:
_model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=False),
    torch.nn.Linear(in_features=1280, out_features=3, bias=True)

).to(device="cuda")

In [ ]:
_model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

In [ ]:
from torchinfo import summary
summary(_model, input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [ ]:
#pause training of features
for params in _model.features.parameters():
    params.requires_grad = False

In [ ]:
from torchinfo import summary
summary(_model, input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [ ]:
_model = _model.to(device="cuda")

In [ ]:
next(_model.parameters()).device

device(type='cuda', index=0)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(_model.parameters(), lr=0.001)

In [ ]:
from Modular_DeepLearning.food101_3_class.train import trainit

trainit(_model, train_load, test_load, loss_fn, optimizer, epochs=5, cala_test_loss_acc=True, device="cuda")

  0%|          | 0/5 [00:00<?, ?it/s]

Train: Loss - 0.6699423748002925 | Accuracy - 0.7576584219932556


 20%|██        | 1/5 [00:12<00:51, 12.94s/it]

Test: Loss - 0.34539300203323364 | Accuracy - 0.9192708730697632
Train: Loss - 0.39744515339253655 | Accuracy - 0.8791373372077942


 40%|████      | 2/5 [00:26<00:39, 13.06s/it]

...

Test: Loss - 0.22256216406822205 | Accuracy - 0.9345238208770752
Train: Loss - 0.2950169282480025 | Accuracy - 0.8986795544624329


 80%|████████  | 4/5 [00:53<00:13, 13.38s/it]

Test: Loss - 0.19611859321594238 | Accuracy - 0.9427083730697632
Train: Loss - 0.2731202082105086 | Accuracy - 0.9039613008499146


100%|██████████| 5/5 [01:06<00:00, 13.26s/it]

Test: Loss - 0.17758896946907043 | Accuracy - 0.9518229365348816


In [ ]:
from PIL import Image
from torchvision import transforms

## hardcoding the class names
# class_names = ['pizza', 'steak', 'sushi']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


img_path = "./Modular_DeepLearning/food101_3_class/premium_photo-1661762555601-47d088a26b50.jpg"
img_path = "./steak.jpg"
img = Image.open(img_path)
img = auto_transform(img).unsqueeze(0)

img = img.to(device)

inference = _model(img)
print(inference)
print(class_names[torch.argmax(inference).item()])
